In [77]:
from ib_insync import *
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
util.startLoop()

In [78]:
ib = IB()
ib.disconnect()
ib.connect('localhost', 7497, clientId=2)

<IB connected to localhost:7497 clientId=2>

In [79]:
# strategy parameters
smaShort = 10
smaLong = 50
freq = "1 min"
units = 1000
contract = Forex("EURUSD")
ib.qualifyContracts(contract)
cfd = CFD("EUR", currency="USD")
ib.qualifyContracts(cfd)
conID = cfd.conId


In [80]:
conID

143916318

In [81]:
def execute_trade(target):
    global currentPos

    # 1. get current Position
    try:
        currentPos = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
        # print(f"currentPos: {currentPos}")
    except:
        currentPos = 0

    # 2. identify required trades
    trades = target - currentPos
    print(f"target: {target}")
    print(f"currentPos: {currentPos}")
    print(f"trades: {trades}")

    # 3. trade execution
    if trades > 0:
        side = "BUY"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)

    elif trades < 0:
        side = "SELL"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    
    else:
        pass

In [82]:
def onBarUpdate(bars, hasNewBar):
    global df, lastBar

    # print(f"bars")
    # print(bars)
    # print()
    if bars[-1].date > lastBar:
        lastBar = bars[-1].date
    
        # data processing
        df = pd.DataFrame(bars)[["date", "open", "high", "low", "close"]].iloc[:-1]
        df.set_index("date", inplace=True)

        # print("original df")
        # display(df)

        # trading strategy
        df = df[["close"]].copy()
    
        df["smaShort"] = df["close"].rolling(smaShort).mean()
        df["smaLong"] = df["close"].rolling(smaLong).mean()
        # print("sma df")
        # display(df)
        df.dropna(inplace=True)
        df["position"] = np.where(df["smaShort"] > df["smaLong"], 1, -1)

        # trading
        target = df["position"][-1] * units
        execute_trade(target=target)

        # display
        clear_output(wait=True)
        display(df)

    else:
        pass


In [ ]:
bars = ib.reqHistoricalData(
        contract,
        endDateTime="",
        durationStr="1 D",
        barSizeSetting=freq,
        whatToShow="MIDPOINT",
        useRTH=True,
        formatDate=2,
        keepUpToDate=True)

# print(bars)
lastBar = bars[-1].date
bars.updateEvent += onBarUpdate

,close,smaShort,smaLong,position
date,,,,
2024-05-14 22:04:00+00:00,1.081685,1.081723,1.081781,-1
2024-05-14 22:05:00+00:00,1.081680,1.081719,1.081777,-1
2024-05-14 22:06:00+00:00,1.081700,1.081718,1.081772,-1
2024-05-14 22:07:00+00:00,1.081715,1.081718,1.081767,-1
2024-05-14 22:08:00+00:00,1.081695,1.081707,1.081761,-1
...,...,...,...,...
2024-05-15 03:30:00+00:00,1.082680,1.082646,1.082555,1
2024-05-15 03:31:00+00:00,1.082715,1.082655,1.082559,1
2024-05-15 03:32:00+00:00,1.082730,1.082666,1.082561,1


Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 10182, reqId 5: Failed to request live updates (disconnected).
Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IBKR and Trader Workstation has been restored - data maintained. The following farms are connected: hfarm; usfarm.nj; usfuture; cashfarm; eufarmnj; usfarm; euhmds; cashhmds; fundfarm; ushmds; secdefil. The following farms are not connected: jfarm.
Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IBKR and Trader Workstation has been restored - data maintained. The following farms are connected: usfarm.nj; usfuture; cashfarm; eufarmnj; usfarm; euhmds; cashhmds; fundfarm; ushmds; secdefil. The following farms are not connected: hfarm; jfarm.
Error 1100, reqId -1: Connectivity between IBKR and Trader Workstation has been lost.
Error

In [74]:
bars.updateEvent -= onBarUpdate

In [ ]:
ib.cancelHistoricalData(bars)

Error 162, reqId 5: Historical Market Data Service error message:API historical data query cancelled: 5


In [52]:
ib.disconnect()